# Iniciando Sessão Spark

In [8]:
from pyspark.sql import SparkSession

# Iniciar a sessão Spark com a configuração do Delta Lake
spark = (SparkSession.builder
    .appName("DeltaLake")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")  # Extensão do Delta
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")  # Catálogo do Delta
    .config("spark.jars", "/usr/local/spark/jars/delta-spark_2.12-3.0.0.jar,/usr/local/spark/jars/delta-storage-3.0.0.jar")  # Caminho dos JARs no Docker
    .config("spark.sql.warehouse.dir", "/home/jovyan/app/data/gold/hive_warehouse/gold_from_delta.db")
    .enableHiveSupport()
    .getOrCreate()) # Se utilizar as limitações de hardware pelo spark, mover para o final
#    .config("spark.executor.cores", "2")  # Limitar o número de núcleos por executor
#    .config("spark.driver.cores", "1")    # Limitar o número de núcleos para o driver
#    .config("spark.executor.memory", "4g")
#    .config("spark.driver.memory", "4g")
#    .config("spark.memory.fraction", "0.3")  # Ajuste para controlar a fração de memória usada pelo Spark
#    .config("spark.memory.storageFraction", "0.2")  # Fração da memória usada para armazenar dados persistentes"""

from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import udf

import time

# Verificar se a sessão Spark foi criada com sucesso
print(spark.version)

3.5.0


# Importando CSV

In [ ]:
df_test = spark.read.csv("/home/jovyan/app/data/raw/temas_ambientais.csv", sep=";", header=True)
df_test.show()

# Salvando Bronze (Distribuída)

In [ ]:
df_test = df_test.dropDuplicates()

In [ ]:
df_test.write.mode("overwrite").format("delta").save("/home/jovyan/app/data/bronze/bronze_distribuida")

# Limpeza e Salvando Silver (Distribuída)

In [3]:
df_test = spark.read.format("delta").load("/home/jovyan/app/data/bronze/bronze_distribuida")
df_test.show(10)

+---+--------------------+-----------+--------------+--------------------+-----------------+--------------------+------------+----------------------------------+---------------------------+---------------------------+---------------------+----------------------+-----------------+-----------------+--------------------+--------------------------------+----------------------+----------------------------+-----------------+---------------+-----------------+---------------------------+----------------------------------------+-----------+-----------------------+
| uf|           municipio|codigo_ibge|area_do_imovel|        registro_car|situacao_cadastro|   condicao_cadastro|area_liquida|area_remanescente_vegetacao_nativa|area_reserva_legal_proposta|area_preservacao_permanente|area_nao_classificada|solicitacao_adesao_pra|         latitude|        longitude|      data_inscricao|data_alteracao_condicao_cadastro|area_rural_consolidada|area_servidao_administrativa|tipo_imovel_rural|modulos_fiscais|

In [4]:
# Criar uma lista com o nome das colunas e o número de nulos em cada uma
null_counts_1 = df_test.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_test.columns]).first().asDict()

# Exibir o resultado
null_counts_1

{'uf': 0,
 'municipio': 0,
 'codigo_ibge': 0,
 'area_do_imovel': 0,
 'registro_car': 0,
 'situacao_cadastro': 0,
 'condicao_cadastro': 0,
 'area_liquida': 60416,
 'area_remanescente_vegetacao_nativa': 60416,
 'area_reserva_legal_proposta': 60416,
 'area_preservacao_permanente': 60416,
 'area_nao_classificada': 60416,
 'solicitacao_adesao_pra': 0,
 'latitude': 0,
 'longitude': 0,
 'data_inscricao': 12,
 'data_alteracao_condicao_cadastro': 5132886,
 'area_rural_consolidada': 60416,
 'area_servidao_administrativa': 60416,
 'tipo_imovel_rural': 0,
 'modulos_fiscais': 0,
 'area_uso_restrito': 60416,
 'area_reserva_legal_averbada': 60416,
 'area_reserva_legal_aprovada_nao_averbada': 60416,
 'area_pousio': 60416,
 'data_ultima_retificacao': 0}

In [ ]:
df_test.printSchema()

In [ ]:
int_columns = ["codigo_ibge"]
float_columns = ["area_do_imovel", "area_liquida", "area_remanescente_vegetacao_nativa", 
                 "area_reserva_legal_proposta", "area_preservacao_permanente", "area_nao_classificada",
                 "latitude", "longitude", "area_rural_consolidada", "area_servidao_administrativa",
                 "modulos_fiscais", "area_uso_restrito", "area_reserva_legal_averbada",
                 "area_reserva_legal_aprovada_nao_averbada", "area_pousio"]
bool_columns = ["solicitacao_adesao_pra"]
date_columns = ["data_inscricao", "data_alteracao_condicao_cadastro", "data_ultima_retificacao"]
text_column = [x for x in df_test.columns if x not in int_columns+float_columns+bool_columns+date_columns]

In [ ]:
# Dados Para Integer
df_test = df_test.withColumn("codigo_ibge", col("codigo_ibge").cast(IntegerType()))

# Dados Para Float
for i in float_columns:
    df_test = df_test.withColumn(i, col(i).cast(FloatType()))

# Dados para Bool
df_test = df_test.withColumn("solicitacao_adesao_pra", 
                             when(col("solicitacao_adesao_pra") == "Sim", True).
                             when(col("solicitacao_adesao_pra") == "Nao", False).
                             otherwise(None))

# Dados para Date
for i in date_columns:
    df_test = df_test.withColumn(i, to_timestamp(regexp_replace(col(i), r"\.\d+$", ""), "yyyy-MM-dd HH:mm:ss"))

df_test.printSchema()

In [ ]:
# Criar uma lista com o nome das colunas e o número de nulos em cada uma
null_counts_2 = df_test.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_test.columns]).first().asDict()

# Exibir o resultado
null_counts_2

In [ ]:
if null_counts_1 == null_counts_2:
    print("Os valores de nulos são os mesmos em ambas as versões.")
else:
    print("Há diferenças nos valores de nulos.")
    # Mostrar as diferenças
    for coluna in null_counts_1.keys():
        if null_counts_1[coluna] != null_counts_2[coluna]:
            print(f"Coluna '{coluna}': null_counts_1 = {null_counts_1[coluna]}, null_counts_2 = {null_counts_2[coluna]}")

# Salvando Silver (Distribuída - Por Estado)

In [ ]:
(df_test.write.mode("overwrite")
              .format("delta")
              .partitionBy("uf")
              .save("/home/jovyan/app/data/silver/silver_distribuida_uf"))

# Querys a partir das Delta Tables

## Funções e Definições

In [9]:
def get_time_delta(query, path):
    start_time = time.time()

    table = spark.sql(query)

    (table.write.mode("overwrite")
                .option("overwriteSchema", "true")
                .format("delta")
                .save(path))
    
    spark.sql(f"""SELECT * FROM delta.`{path}`""").show()

    end_time = time.time()
    
    tempo_query = end_time - start_time

    return tempo_query

def is_point_in_path(x: float, y: float, poly: list[tuple[float, float]]) -> bool:
    """Determine if the point is on the path, corner, or boundary of the polygon

    Args:
    x -- The x coordinates of point.
    y -- The y coordinates of point.
    poly -- a list of tuples [(x, y), (x, y), ...]

    Returns:
        True if the point is in the path or is a corner or on the boundary"""
    
    c = False

    for i in range(len(poly)):
        ax, ay = poly[i]
        bx, by = poly[i - 1]
        if (x == ax) and (y == ay):
            # point is a corner
            return True
        if (ay > y) != (by > y):
            slope = (x - ax) * (by - ay) - (bx - ax) * (y - ay)
            if slope == 0:
                # point is on boundary
                return True
            if (slope < 0) != (by < ay):
                c = not c

    return c

# Registrar a UDF no Spark
poly = [(-53.5325072, -19.4632582), (-51.0495971, -19.1625841), (-51.3734501, -16.1924262), (-53.8181518, -16.4010783), (-53.5325072, -19.4632582)]

is_point_in_polygon_udf = udf(lambda x, y: is_point_in_path(x, y, poly), BooleanType())

spark.udf.register("is_point_in_polygon", is_point_in_polygon_udf)

## Consulta 1

Recupere a soma de área (em hectares) para todas as propriedades agrícolas que pertencem ao MS e MT. Ordene os resultados em ordem decrescente.

In [10]:
query = """
SELECT 
   uf,
   FORMAT_NUMBER(SUM(area_do_imovel), 3) as soma_area_hectares

FROM delta.`/home/jovyan/app/data/silver/silver_distribuida_uf`

WHERE uf IN ('MS', 'MT')

GROUP BY uf

ORDER BY soma_area_hectares DESC

"""

tempo_delta_1 = get_time_delta(query, "/home/jovyan/app/data/gold/delta/query_1")
tempo_delta_1

+---+------------------+
| uf|soma_area_hectares|
+---+------------------+
| MT|    84,587,089.218|
| MS|    36,321,882.660|
+---+------------------+



11.146804094314575

## Consulta 2
Filtre todas as propriedades que pertecem a região sudeste. 

In [11]:
query = """
SELECT
   uf,
   codigo_ibge

FROM delta.`/home/jovyan/app/data/silver/silver_distribuida_uf`

WHERE uf IN ('SP','MG','ES','RJ')
"""

tempo_delta_2 = get_time_delta(query, "/home/jovyan/app/data/gold/delta/query_2")
tempo_delta_2

+---+-----------+
| uf|codigo_ibge|
+---+-----------+
| ES|    3202108|
| ES|    3200359|
| ES|    3200904|
| ES|    3203353|
| ES|    3200607|
| ES|    3202652|
| ES|    3203056|
| ES|    3202652|
| ES|    3203056|
| ES|    3201605|
| ES|    3200169|
| ES|    3203056|
| ES|    3200102|
| ES|    3203007|
| ES|    3201001|
| ES|    3202801|
| ES|    3205002|
| ES|    3201902|
| ES|    3203056|
| ES|    3203205|
+---+-----------+
only showing top 20 rows



8.41820240020752

## Consulta 3
Recupere todas as propriedades rurais que estão localizadas dentro de uma área geográfica específica delimitada por um polígono. Este polígono é descrito pelas seguintes coordenadas: POLYGON ((-53.5325072 -19.4632582, -51.0495971 -19.1625841, -51.3734501 -16.1924262, -53.8181518 -16.4010783, -53.5325072 -19.4632582))
Verificar se um centroide está dentro dos pontos informados (ou mais).

In [12]:
query = """
SELECT
    codigo_ibge,
    latitude,
    longitude,
    is_point_in_polygon(longitude, latitude) AS inside_polygon

FROM delta.`/home/jovyan/app/data/silver/silver_distribuida_uf`

WHERE is_point_in_polygon(longitude, latitude) = True
"""

tempo_delta_3 = get_time_delta(query, "/home/jovyan/app/data/gold/delta/query_3")
tempo_delta_3

+-----------+----------+----------+--------------+
|codigo_ibge|  latitude| longitude|inside_polygon|
+-----------+----------+----------+--------------+
|    5211909| -18.03174| -52.08481|          true|
|    5201454|-18.287004|-51.305717|          true|
|    5213103|-17.791296|-52.508606|          true|
|    5211909|-17.901014|-52.022953|          true|
|    5204409|-17.068356|-51.634693|          true|
|    5211909|-18.044683|-51.519875|          true|
|    5213103|-17.640839| -52.49887|          true|
|    5219407|-17.562614|  -53.2227|          true|
|    5211909|-18.050959|-51.590702|          true|
|    5215652| -16.64014|-51.353027|          true|
|    5204409|-16.975107| -51.85558|          true|
|    5211305|-18.557215|-51.539852|          true|
|    5219407|-17.350077| -53.08434|          true|
|    5218102|-17.293552| -52.66369|          true|
|    5213103|-17.213865|  -52.7558|          true|
|    5201454|-18.237848|-51.257576|          true|
|    5204409|-16.991863|-52.290

21.601632356643677

## Consulta 4
Calcule quantas propriedades foram cadastradas por ano. Apresente os resultados em ordem cronológica.

In [13]:
query = """
SELECT
   YEAR(data_inscricao) AS ano_inscricao,
   COUNT(*) AS propriedade_cadastradas

FROM delta.`/home/jovyan/app/data/silver/silver_distribuida_uf`

GROUP BY ano_inscricao

ORDER BY ano_inscricao
"""

tempo_delta_4 = get_time_delta(query, "/home/jovyan/app/data/gold/delta/query_4")
tempo_delta_4

+-------------+-----------------------+
|ano_inscricao|propriedade_cadastradas|
+-------------+-----------------------+
|         NULL|                     12|
|         2013|                    743|
|         2014|                 227468|
|         2015|                1240176|
|         2016|                1828786|
|         2017|                 794777|
|         2018|                 724856|
|         2019|                 746570|
|         2020|                 463322|
|         2021|                 443942|
|         2022|                 368448|
+-------------+-----------------------+



12.626770973205566

## Consulta 5
Calcule o percentual médio de área remanescente de vegetação nativa em comparação a área total da propriedade

In [14]:
query = """
SELECT
   codigo_ibge,
   area_do_imovel,
   area_remanescente_vegetacao_nativa,
   FORMAT_NUMBER(area_remanescente_vegetacao_nativa / area_do_imovel * 100, 2) as percentual_area_nativa

FROM delta.`/home/jovyan/app/data/silver/silver_distribuida_uf`
"""

tempo_delta_5 = get_time_delta(query, "/home/jovyan/app/data/gold/delta/query_5")
tempo_delta_5

+-----------+--------------+----------------------------------+----------------------+
|codigo_ibge|area_do_imovel|area_remanescente_vegetacao_nativa|percentual_area_nativa|
+-----------+--------------+----------------------------------+----------------------+
|    2802106|        70.852|                               0.0|                  0.00|
|    2800506|        1.8226|                               0.0|                  0.00|
|    2802908|        2.9685|                         1.2656308|                 42.64|
|    2805307|       52.0451|                               0.0|                  0.00|
|    2804201|       33.0333|                               0.0|                  0.00|
|    2804102|        3.2018|                               0.0|                  0.00|
|    2807501|        7.1537|                               0.0|                  0.00|
|    2805604|        35.262|                               0.0|                  0.00|
|    2804458|        1.7241|               

19.61841320991516

## Consulta 6
Construa uma consulta que mostre a contagem de propriedades rurais por estado.

* AST – Assentamentos
* CAR – Cadastro Ambiental Rural
* CCU – Contrato de Concessão de Uso
* CDRU – Concessão de Direito Real de Uso
* DD – Diretoria de Desenvolvimento
* IRU – Imóveis Rurais
* MLC – Módulo Lote CAR
* PA – Projeto de Assentamento
* PCT – Povos e Comunidades Tradicionais
* PRA – Programa de Regularização Ambiental
* PNRA – Programa Nacional de Reforma Agrária
* SICAR – Sistema de Cadastro Ambiental Rural
* SR – Superintendência Regional
* TCMS – Termo de Compromisso de Manutenção de Sigilo
* TD – Título Definitivo
* TED – Termo de Execução Descentralizada
* URL – Uniform Resource Locator

In [15]:
query = """
SELECT
DISTINCT tipo_imovel_rural
                    
FROM delta.`/home/jovyan/app/data/silver/silver_distribuida_uf`
"""

spark.sql(query).show()

+-----------------+
|tipo_imovel_rural|
+-----------------+
|              AST|
|              IRU|
|              PCT|
+-----------------+



In [16]:
query = """
SELECT
    uf,
    COUNT(tipo_imovel_rural) AS count_prop_rural

FROM delta.`/home/jovyan/app/data/silver/silver_distribuida_uf`

WHERE tipo_imovel_rural = 'IRU'

GROUP BY uf

ORDER BY count_prop_rural DESC
"""

tempo_delta_6 = get_time_delta(query, "/home/jovyan/app/data/gold/delta/query_6")
tempo_delta_6

+---+----------------+
| uf|count_prop_rural|
+---+----------------+
| BA|         1011503|
| MG|          988921|
| RS|          606013|
| PR|          495650|
| SP|          406885|
| SC|          374768|
| PE|          338392|
| CE|          314168|
| PA|          278097|
| MA|          272010|
| PI|          254577|
| GO|          198524|
| PB|          170484|
| MT|          169775|
| RO|          147099|
| AL|          113222|
| ES|          108848|
| SE|           94203|
| RN|           88788|
| TO|           84990|
+---+----------------+
only showing top 20 rows



11.632027626037598

## Consulta 7
Veja qual é a maior propriedade entre todas e calcule a distância entre ela e Brasília. Utilize a coordenada de centródide da propriedade para calcular a distância entre ela e Brasília. Coordenadas de Brasília: -15.796943053171708, -47.891638482569476

In [17]:
query = """
SELECT
    max_area,
    latitude,
    longitude,
    FORMAT_NUMBER(
        SQRT(
            POWER(latitude - (-15.796943053171708), 2) + 
            POWER(longitude - (-47.891638482569476), 2) 
            )* 111, 2
        )  AS distancia_brasilia

FROM (

    SELECT 
        latitude, 
        longitude, 
        area_do_imovel AS max_area

    FROM delta.`/home/jovyan/app/data/silver/silver_distribuida_uf`

    ORDER BY area_do_imovel DESC

    LIMIT 1
)
"""

tempo_delta_7 = get_time_delta(query, "/home/jovyan/app/data/gold/delta/query_7")
tempo_delta_7

+---------+---------+---------+------------------+
| max_area| latitude|longitude|distancia_brasilia|
+---------+---------+---------+------------------+
|2420078.8|-5.475619|-68.88819|          2,596.98|
+---------+---------+---------+------------------+



11.845029592514038

## Consulta 8
Faça a média de área entre todas as propriedades. Calcule quantas propriedades por estado, estão acima da média.

In [18]:
query = """
SELECT DISTINCT
    uf,
    COUNT(registro_car) AS qnt_propriedades,
    AVG(area_do_imovel) AS media_area

FROM delta.`/home/jovyan/app/data/silver/silver_distribuida_uf`

GROUP BY uf

HAVING media_area > (SELECT AVG(area_do_imovel) FROM delta.`/home/jovyan/app/data/silver/silver_distribuida_uf`);
"""

tempo_delta_8 = get_time_delta(query, "/home/jovyan/app/data/gold/delta/query_8")
tempo_delta_8

+---+----------------+------------------+
| uf|qnt_propriedades|        media_area|
+---+----------------+------------------+
| PA|          279598| 293.7753804628661|
| MA|          273642| 112.5997265257679|
| GO|          199690|164.78160679766881|
| MT|          170205| 496.9718234972293|
| RO|          147286|108.26146315792728|
| TO|           85537| 293.6158931536979|
| AM|           73642| 968.9693483889848|
| MS|           79914| 454.5121337998023|
| AC|           45141| 306.8832352226421|
| RR|           22687| 390.4115962484108|
| AP|           10116|511.92016272641627|
+---+----------------+------------------+



19.178141117095947

## Tempo Total Delta Tables

In [19]:
total_delta = tempo_delta_1 + tempo_delta_2 + tempo_delta_3 + tempo_delta_4 + tempo_delta_5 + tempo_delta_6 + tempo_delta_7 + tempo_delta_8
total_delta

116.06702136993408

# Querys a partir da Hive Metastore

## Salvando Gold (Distribuída)

In [20]:
spark.sql("CREATE SCHEMA IF NOT EXISTS gold_from_delta")

DataFrame[]

In [21]:
df_test = spark.read.format("delta").load("/home/jovyan/app/data/silver/silver_distribuida_uf")

df_test.write.format("delta").option("overwriteSchema", "true").mode("overwrite").partitionBy("uf").saveAsTable("gold_from_delta.gold_distribuida_uf")

## Funções e Definições

In [22]:
def get_time_hive(query, table_name):
    start_time = time.time()

    table = spark.sql(query)

    (table.write.format("delta")
                .mode("overwrite")
                .option("overwriteSchema", "true")
                .saveAsTable(f"gold_from_delta.{table_name}"))
    
    spark.sql(f"""SELECT * FROM gold_from_delta.{table_name}""").show()

    end_time = time.time()
    
    tempo_query = end_time - start_time

    return tempo_query

## Consulta 1

Recupere a soma de área (em hectares) para todas as propriedades agrícolas que pertencem ao MS e MT. Ordene os resultados em ordem decrescente.

In [23]:
query = """
SELECT 
   uf,
   FORMAT_NUMBER(SUM(area_do_imovel), 3) as soma_area_hectares

FROM gold_from_delta.gold_distribuida_uf

WHERE uf IN ('MS', 'MT')

GROUP BY uf

ORDER BY soma_area_hectares DESC

"""

tempo_hive_1 = get_time_hive(query, "query_1")
tempo_hive_1

+---+------------------+
| uf|soma_area_hectares|
+---+------------------+
| MT|    84,587,089.218|
| MS|    36,321,882.660|
+---+------------------+



5.736438274383545

## Consulta 2
Filtre todas as propriedades que pertecem a região sudeste. 

In [24]:
query = """
SELECT
   uf,
   codigo_ibge

FROM gold_from_delta.gold_distribuida_uf

WHERE uf IN ('SP','MG','ES','RJ')
"""

tempo_hive_2 = get_time_hive(query, "query_2")
tempo_hive_2

+---+-----------+
| uf|codigo_ibge|
+---+-----------+
| MG|    3136801|
| MG|    3136520|
| MG|    3145604|
| MG|    3171808|
| MG|    3104007|
| MG|    3123908|
| MG|    3114600|
| MG|    3155603|
| MG|    3171071|
| MG|    3112307|
| MG|    3112406|
| MG|    3169307|
| MG|    3108503|
| MG|    3139904|
| MG|    3119906|
| MG|    3170206|
| MG|    3165578|
| MG|    3158201|
| MG|    3104452|
| MG|    3131604|
+---+-----------+
only showing top 20 rows



6.054698705673218

## Consulta 3
Recupere todas as propriedades rurais que estão localizadas dentro de uma área geográfica específica delimitada por um polígono. Este polígono é descrito pelas seguintes coordenadas: POLYGON ((-53.5325072 -19.4632582, -51.0495971 -19.1625841, -51.3734501 -16.1924262, -53.8181518 -16.4010783, -53.5325072 -19.4632582))
Verificar se um centroide está dentro dos pontos informados (ou mais).

In [25]:
query = """
SELECT
    codigo_ibge,
    latitude,
    longitude,
    is_point_in_polygon(longitude, latitude) AS inside_polygon

FROM gold_from_delta.gold_distribuida_uf

WHERE is_point_in_polygon(longitude, latitude) = True
"""

tempo_hive_3 = get_time_hive(query, "query_3")
tempo_hive_3

+-----------+----------+----------+--------------+
|codigo_ibge|  latitude| longitude|inside_polygon|
+-----------+----------+----------+--------------+
|    5100300|-17.226484|-53.723015|          true|
|    5108204|-16.369793| -52.97373|          true|
|    5100300|-17.202026|-53.406887|          true|
|    5100300| -17.36272|-53.273006|          true|
|    5100607|-17.924597|-53.304733|          true|
|    5100409|-16.595022|-53.215626|          true|
|    5107198|-16.566946|-52.681526|          true|
|    5100409|-16.970575| -53.64971|          true|
|    5100300|-16.960646|-53.144123|          true|
|    5108204|-16.450071|-53.159634|          true|
|    5108204|-16.379072|-53.078815|          true|
|    5100300|-17.759611|-53.631786|          true|
|    5100607|-17.894436|-53.370655|          true|
|    5108204|-16.350658|-52.798054|          true|
|    5104203|-16.399292|-53.751965|          true|
|    5100300|-17.631042|-53.534096|          true|
|    5100409| -16.69768|-53.487

16.425443172454834

## Consulta 4
Calcule quantas propriedades foram cadastradas por ano. Apresente os resultados em ordem cronológica.

In [26]:
query = """
SELECT
   YEAR(data_inscricao) AS ano_inscricao,
   COUNT(*) AS propriedade_cadastradas

FROM gold_from_delta.gold_distribuida_uf

GROUP BY ano_inscricao

ORDER BY ano_inscricao
"""

tempo_hive_4 = get_time_hive(query, "query_4")
tempo_hive_4

+-------------+-----------------------+
|ano_inscricao|propriedade_cadastradas|
+-------------+-----------------------+
|         NULL|                     12|
|         2013|                    743|
|         2014|                 227468|
|         2015|                1240176|
|         2016|                1828786|
|         2017|                 794777|
|         2018|                 724856|
|         2019|                 746570|
|         2020|                 463322|
|         2021|                 443942|
|         2022|                 368448|
+-------------+-----------------------+



7.400269269943237

## Consulta 5
Calcule o percentual médio de área remanescente de vegetação nativa em comparação a área total da propriedade

In [27]:
query = """
SELECT
   codigo_ibge,
   area_do_imovel,
   area_remanescente_vegetacao_nativa,
   FORMAT_NUMBER(area_remanescente_vegetacao_nativa / area_do_imovel * 100, 2) as percentual_area_nativa

FROM gold_from_delta.gold_distribuida_uf
"""

tempo_hive_5 = get_time_hive(query, "query_5")
tempo_hive_5

+-----------+--------------+----------------------------------+----------------------+
|codigo_ibge|area_do_imovel|area_remanescente_vegetacao_nativa|percentual_area_nativa|
+-----------+--------------+----------------------------------+----------------------+
|    3136801|      119.0454|                               0.0|                  0.00|
|    3136520|       34.5053|                               0.0|                  0.00|
|    3145604|        3.5613|                               0.0|                  0.00|
|    3171808|       22.0177|                         1.1560909|                  5.25|
|    3104007|       25.9163|                         5.0354385|                 19.43|
|    3123908|        48.444|                               0.0|                  0.00|
|    3114600|       35.5403|                         7.0675855|                 19.89|
|    3155603|        1.8732|                               0.0|                  0.00|
|    3171071|       20.7036|               

12.504667520523071

## Consulta 6
Construa uma consulta que mostre a contagem de propriedades rurais por estado.

In [28]:
query = """
SELECT
    uf,
    COUNT(tipo_imovel_rural) AS count_prop_rural

FROM gold_from_delta.gold_distribuida_uf

WHERE tipo_imovel_rural = 'IRU'

GROUP BY uf

ORDER BY count_prop_rural DESC
"""

tempo_hive_6 = get_time_hive(query, "query_6")
tempo_hive_6

+---+----------------+
| uf|count_prop_rural|
+---+----------------+
| BA|         1011503|
| MG|          988921|
| RS|          606013|
| PR|          495650|
| SP|          406885|
| SC|          374768|
| PE|          338392|
| CE|          314168|
| PA|          278097|
| MA|          272010|
| PI|          254577|
| GO|          198524|
| PB|          170484|
| MT|          169775|
| RO|          147099|
| AL|          113222|
| ES|          108848|
| SE|           94203|
| RN|           88788|
| TO|           84990|
+---+----------------+
only showing top 20 rows



6.519151926040649

## Consulta 7
Veja qual é a maior propriedade entre todas e calcule a distância entre ela e Brasília. Utilize a coordenada de centródide da propriedade para calcular a distância entre ela e Brasília. Coordenadas de Brasília: -15.796943053171708, -47.891638482569476

In [29]:
query = """
SELECT
    max_area,
    latitude,
    longitude,
    FORMAT_NUMBER(
        SQRT(
            POWER(latitude - (-15.796943053171708), 2) + 
            POWER(longitude - (-47.891638482569476), 2) 
            )* 111, 2
        )  AS distancia_brasilia

FROM (

    SELECT 
        latitude, 
        longitude, 
        area_do_imovel AS max_area

    FROM gold_from_delta.gold_distribuida_uf

    ORDER BY area_do_imovel DESC

    LIMIT 1
)
"""

tempo_hive_7 = get_time_hive(query, "query_7")
tempo_hive_7

+---------+---------+---------+------------------+
| max_area| latitude|longitude|distancia_brasilia|
+---------+---------+---------+------------------+
|2420078.8|-5.475619|-68.88819|          2,596.98|
+---------+---------+---------+------------------+



6.637845039367676

## Consulta 8
Faça a média de área entre todas as propriedades. Calcule quantas propriedades por estado, estão acima da média.

In [30]:
query = """
SELECT DISTINCT
    uf,
    COUNT(registro_car) AS qnt_propriedades,
    AVG(area_do_imovel) AS media_area

FROM gold_from_delta.gold_distribuida_uf

GROUP BY uf

HAVING media_area > (SELECT AVG(area_do_imovel) FROM gold_from_delta.gold_distribuida_uf);
"""

tempo_hive_8 = get_time_hive(query, "query_8")
tempo_hive_8

+---+----------------+------------------+
| uf|qnt_propriedades|        media_area|
+---+----------------+------------------+
| PA|          279598| 293.7753804628661|
| MT|          170205| 496.9718234972293|
| MA|          273642|112.59972652576785|
| RO|          147286|108.26146315792728|
| AM|           73642| 968.9693483889848|
| GO|          199690|164.78160679766881|
| TO|           85537| 293.6158931536979|
| MS|           79914| 454.5121337998023|
| AC|           45141| 306.8832352226421|
| RR|           22687| 390.4115962484108|
| AP|           10116|511.92016272641627|
+---+----------------+------------------+



9.380352973937988

## Tempo Total Hive Metastore

In [31]:
total_hive = tempo_hive_1 + tempo_hive_2 + tempo_hive_3 + tempo_hive_4 + tempo_hive_5 + tempo_hive_6 + tempo_hive_7 + tempo_hive_8
total_hive

70.65886688232422

In [35]:
spark.sql("SHOW SCHEMAS").show()

+---------------+
|      namespace|
+---------------+
|        default|
|gold_from_delta|
+---------------+



In [36]:
spark.sql("SHOW TABLES IN gold_from_delta").show()

+---------------+-------------------+-----------+
|      namespace|          tableName|isTemporary|
+---------------+-------------------+-----------+
|gold_from_delta|gold_distribuida_uf|      false|
|gold_from_delta|            query_1|      false|
|gold_from_delta|            query_2|      false|
|gold_from_delta|            query_3|      false|
|gold_from_delta|            query_4|      false|
|gold_from_delta|            query_5|      false|
|gold_from_delta|            query_6|      false|
|gold_from_delta|            query_7|      false|
|gold_from_delta|            query_8|      false|
+---------------+-------------------+-----------+



In [34]:
# Deletar schema e todos os objetos
# spark.sql("DROP DATABASE IF EXISTS gold_from_delta CASCADE")

DataFrame[]